# **4.0 Lets add Memory and Context to our Agent**

## **Introduction**

In this tutorial, we’ll build an interactive AI agent using **LangChain**, **LangGraph**, and **Memory**. The agent will:

- **Remember previous interactions** for more context-aware responses.
- Use **LangGraph** to manage decision-making and visualize workflows.
- Leverage **LangChain tools** to interact with network devices and analyze logs.

By the end, you’ll have a fully functional AI agent that processes commands, retains context, and helps with network troubleshooting.


## **Workshop Outline**

1. **Setup**: Install required libraries and configure the environment.
2. **LangChain Agent**: Initialize the agent with memory.
3. **LangGraph**: Visualize decision-making and task flow.
4. **Tool Integration**: Implement network command execution and log analysis.
5. **Testing & Visualization**: Test the agent and visualize its decision-making.


### **Complete the Following Pre-Requisites**  
1. Select the kernel: **"Python(ai-agent)"**  
2. Perform **Clear all outputs**

### Step 1: Warning Control

In this step, we import the `warnings` module and suppress any warning messages that may appear during code execution.

- **`import warnings`**: Imports the Python `warnings` module to handle warning messages.
- **`warnings.filterwarnings('ignore')`**: Instructs Python to ignore all warnings, commonly used to keep the output clean when warnings are not critical.

This helps reduce clutter in the output, ensuring that only important results are shown.

In [ ]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

## Step 2: Importing Required Modules and Initializing Memory

In this step, we import various modules required for building the agent and initialize in-memory storage for tracking state and memory.

- **`from langgraph.graph import StateGraph, END`**: Imports `StateGraph` for managing decision flows and `END` for terminating the flow.
- **`from langchain_openai import ChatOpenAI`**: Imports the `ChatOpenAI` model to interact with the language model.
- **`from langgraph.checkpoint.sqlite import SqliteSaver`**: Imports `SqliteSaver` for saving the state in SQLite, allowing the agent to maintain memory.
- **`from contextlib import ExitStack`**: Used to manage context for resources like memory or databases.
- **`from dotenv import load_dotenv`**: Loads environment variables from a `.env` file.

The **`ExitStack()`** context manager is used to initialize **in-memory SQLite storage** with **`SqliteSaver`**, which helps in saving and retrieving memory states during the agent's interactions.

In [ ]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults


from langgraph.checkpoint.sqlite import SqliteSaver
from contextlib import ExitStack

from dotenv import load_dotenv
_ = load_dotenv()

stack = ExitStack()
memory = stack.enter_context(SqliteSaver.from_conn_string(":memory:"))

## Step 3: Initializing the Tool

In this step, we initialize a tool for search functionality and print its type and name.

- **`tool = TavilySearchResults(max_results=4)`**: Initializes the `TavilySearchResults` tool with a limit of 4 results per query.
- **`print(type(tool))`**: Prints the type of the initialized tool to confirm its class.
- **`print(tool.name)`**: Prints the name of the tool to confirm its functionality.

This step allows us to verify that the tool is set up correctly with the sp

In [ ]:
tool = TavilySearchResults(max_results=4) #increased number of results

## Step 4: Defining the Agent's State

In this step, we define a class to represent the state of the agent. The state includes the messages that are exchanged during the interaction.

- **`class AgentState(TypedDict)`**: Defines a `TypedDict` class named `AgentState`, which is a dictionary-like object that includes the types for the state.
- **`messages: Annotated[list[AnyMessage], operator.add]`**: Specifies that the `messages` field will store a list of `AnyMessage` objects, and it uses the `operator.add` function to potentially modify or extend the list of messages.

This step helps to structure and manage the agent’s state, including handling the flow of messages throughout the interaction.

In [ ]:
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]

## Step 5: Defining the Agent Class

We define the `Agent` class to manage the agent’s workflow, interactions, and actions:

- **`__init__(self, model, tools, checkpointer, system="")`**: Initializes the agent with a language model, tools, and state graph.
  - **`graph.add_node()`**: Adds nodes for interacting with the model and performing actions.
  - **`graph.add_conditional_edges()`**: Determines whether to take action or finish.

- **`exists_action(self, state)`**: Checks if an action is required based on tool calls.
- **`call_openai(self, state)`**: Invokes the language model with current messages.
- **`take_action(self, state)`**: Executes actions based on tool calls and returns results.

This class manages input, decision-making, and tool interaction.

> Note: in `take_action` below, some logic was added to cover the case that the LLM returned a non-existent tool name. Even with function calling, LLMs can still occasionally hallucinate. Note that all that is done is instructing the LLM to try again! An advantage of an agentic organization.

In [ ]:
class Agent:

    def __init__(self, model, tools, checkpointer, system=""):
        self.system = system
        graph = StateGraph(AgentState)
        graph.add_node("llm", self.call_openai)
        graph.add_node("action", self.take_action)
        graph.add_conditional_edges(
            "llm",
            self.exists_action,
            {True: "action", False: END}
        )
        graph.add_edge("action", "llm")
        graph.set_entry_point("llm")
        self.graph = graph.compile(checkpointer=checkpointer)
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)

    def exists_action(self, state: AgentState):
        result = state['messages'][-1]
        return len(result.tool_calls) > 0

    def call_openai(self, state: AgentState):
        messages = state['messages']
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {'messages': [message]}

    def take_action(self, state: AgentState):
        tool_calls = state['messages'][-1].tool_calls
        results = []
        for t in tool_calls:
            print(f"Calling: {t}")
            if not t['name'] in self.tools:      # check for bad tool name from LLM
                print("\n ....bad tool name....")
                result = "bad tool name, retry"  # instruct LLM to retry if bad
            else:
                result = self.tools[t['name']].invoke(t['args'])
            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
        print("Back to the model!")
        return {'messages': results}

## Step 6: Defining the Agent's Prompt and Initializing the Agent

In this step, we define the agent's prompt and initialize the agent with the model and tools.

- **`prompt`**: Defines the system message, guiding the agent's behavior for research tasks. It tells the agent when and how to use the search engine for information retrieval.
  
- **`model = ChatOpenAI(model="gpt-3.5-turbo")`**: Initializes the language model (GPT-3.5).

- **`abot = Agent(model, [tool], system=prompt, checkpointer=memory)`**: Initializes the `Agent` with the model, tools, system prompt, a

In [ ]:
prompt = """You are a smart research assistant. Use the search engine to look up information. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""

model = ChatOpenAI(model="gpt-3.5-turbo")  
abot = Agent(model, [tool], system=prompt,checkpointer=memory)

## Step 7: Displaying the Agent's Workflow

This step adds a directory to the system’s `PATH` and visualizes the agent's workflow graph.

In [ ]:
import os
os.environ["PATH"] += os.pathsep + "/opt/homebrew/bin"

from IPython.display import Image

Image(abot.graph.get_graph().draw_png())

## Step 8: Sending Messages to the Agent

In this step, we send a message to the agent and process the response.

- We define a `messages` list with the user input.
- A `thread` is created to maintain the context.
- The agent processes the input using the `.stream()` method and the response is printed.

This allows real-time interaction with the agent.


In [ ]:
messages = [HumanMessage(content="Whats the currency of Lisbon")]
thread = {"configurable": {"thread_id": "1"}}

for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v['messages'])

In [ ]:
messages = [HumanMessage(content="What about in India?")]
thread = {"configurable": {"thread_id": "1"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)

In [ ]:
messages = [HumanMessage(content="Which one is more stronger?")]
thread = {"configurable": {"thread_id": "1"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)

## Now lets try with a different thread id

In [ ]:
messages = [HumanMessage(content="Which one is stronger?")]
thread = {"configurable": {"thread_id": "2"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)

## Conclusion

In this tutorial, we built an interactive AI agent using **LangChain**, **LangGraph**, and **Memory**. We:

- Initialized the agent with a language model and tools.
- Implemented memory to track previous interactions.
- Used LangGraph to visualize the agent's decision-making process.
- Interacted with the agent in real-time, sending messages and processing responses.

By the end of this tutorial, you now have a fully functional AI agent that can make decisions, interact with external tools, and retain context across conversations. You can extend this agent with additional tools, improve its decision-making capabilities, and further customize its behavior to fit specific use cases.